# Analyzing the MSTIS simulation

Included in this notebook:

* Opening files for analysis
* Rates, fluxes, total crossing probabilities, and condition transition probabilities
* Per-ensemble properties such as path length distributions and interface crossing probabilities
* Move scheme analysis
* Replica exchange analysis
* Replica move history tree visualization
* Replaying the simulation
* MORE TO COME! Like free energy projections, path density plots, and more

In [1]:
import logging
reload(logging)
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.info('Info logging activated')
init_log = logging.getLogger('openpathsampling.initialization')
init_log.setLevel(logging.CRITICAL)

INFO:root:Info logging activated


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import openpathsampling as paths
import numpy as np

The optimum way to use storage depends on whether you're doing production or analysis. For analysis, you should open the file as an `AnalysisStorage` object. This makes the analysis much faster.

In [ ]:
%%time
storage = paths.Storage("mstis.nc", "r")

INFO:openpathsampling.netcdfplus.netcdfplus:Open existing netCDF file 'mstis.nc' for reading - reading from existing file


In [ ]:
storage.set_caching_mode('analysis')

In [5]:
paths.AnalysisStorage.cache_for_analysis(storage)

INFO:openpathsampling.storage.storage:store.cvs[CollectiveVariable] in 12 ms
INFO:openpathsampling.storage.storage:store.samples[Sample] in 2266 ms
INFO:openpathsampling.storage.storage:store.samplesets[SampleSet] in 377 ms
INFO:openpathsampling.storage.storage:store.volumes[Volume] in 1 ms
INFO:openpathsampling.storage.storage:store.ensembles[Ensemble] in 1 ms
INFO:openpathsampling.storage.storage:store.pathmovers[PathMover] in 390 ms
INFO:openpathsampling.storage.storage:store.pathmovechanges[PathMoveChange] in 783 ms


ValueError: badly formed hexadecimal UUID string

In [8]:
%%time
storage.cvs.cache_all()

CPU times: user 3.02 ms, sys: 1.28 ms, total: 4.3 ms
Wall time: 2.82 ms


In [4]:
%%time
for cv, (cv_store, cv_store_idx) in storage.snapshots.cv_list.items():
    cv_store.cache.load_max()

CPU times: user 363 ms, sys: 37.3 ms, total: 400 ms
Wall time: 400 ms


In [7]:
%%time
storage.samples.cache_all()

CPU times: user 2.26 s, sys: 40.7 ms, total: 2.3 s
Wall time: 2.3 s


In [8]:
%%time
storage.samplesets.cache_all()

CPU times: user 360 ms, sys: 7.94 ms, total: 368 ms
Wall time: 367 ms


In [9]:
%%time
storage.volumes.cache_all()

CPU times: user 1.87 ms, sys: 948 µs, total: 2.82 ms
Wall time: 1.78 ms


In [ ]:
%%time


In [ ]:
mstis = storage.networks.load(0)

In [ ]:
for store in storage.objects.values():
    print store.name, len(store.index), len(store), store.__class__.__name__

## Reaction rates

TIS methods are especially good at determining reaction rates, and OPS makes it extremely easy to obtain the rate from a TIS network.

Note that, although you can get the rate directly, it is very important to look at other results of the sampling (illustrated in this notebook and in notebooks referred to herein) in order to check the validity of the rates you obtain.

By default, the built-in analysis calculates histograms the maximum value of some order parameter and the pathlength of every sampled ensemble. You can add other things to this list as well, but you must always specify histogram parameters for these two. The pathlength is in units of frames.

In [ ]:
mstis.hist_args['max_lambda'] = { 'bin_width' : 0.02, 'bin_range' : (0.0, 0.5) }
mstis.hist_args['pathlength'] = { 'bin_width' : 5, 'bin_range' : (0, 150) }

In [ ]:
cv = storage.cvs[0]

In [ ]:
storage.trajectories[200]

In [ ]:
len(storage.snapshots.index)

In [ ]:
len(storage.snapshot0)

In [ ]:
%%time
cv(storage.trajectories[370])

In [ ]:
sum(map(len, storage.trajectories))

In [ ]:
q = storage.snapshots.all()

In [ ]:
len(q)

In [ ]:
for cv in storage.cvs:
    store = cv._store_dict.value_store
    store.cache.load_max()
    print store.allow_partial
    print store.chunksize
    print cv.diskcache_enabled
    print cv.cv_time_reversible

In [ ]:
cv._store_dict.value_store.cache.load_max()

In [ ]:
for x in range(len(q[0:19999])):
    y = list.__getitem__(q, x)
    print x, cv._store_dict.value_store.cache[x]

In [ ]:
%%prun
_ = cv(q[0:9600])

In [ ]:
for cv in storage.cvs:
    cd = cv._store_dict.value_store.cache._chunkdict
    print cd.keys()
    print map(len, cd.values())

In [ ]:
cd = cv._store_dict.value_store.cache._chunkdict
print cd.keys()
print map(len, cd.values())

In [ ]:
xx = 159
wi = None
print cd[xx][0:wi]
print cv._store_dict.value_store.vars['value'][xx*100:xx*100 + (wi or 100)]

storage.cvs[0]

In [ ]:
cv._store_dict.value_store.cache.load_max()

In [ ]:
%%time
cv(q[0:1000])

In [ ]:
%%time
mstis.rate_matrix(storage.steps)

In [ ]:
%%prun -s cumtime
for t in storage.trajectories:
    print cv(t)

In [ ]:
%%time
for t in storage.trajectories:
    q = cv(t)

In [ ]:
%%time
for s in storage.snapshots.all().as_proxies():
    cv(s)

In [ ]:
%%time
ll = 0
for step in storage.steps:
    sset = step.active # take the sampleset after the move
    for sample in sset:
        ll += len(sample.trajectory)
#        print storage.idx(sample.ensemble),
        
        cv(sample)
        
print ll

In [ ]:
%%prun -s cumtime
mstis.rate_matrix(storage, force=True)

The self-rates (the rate of returning the to initial state) are undefined, and return not-a-number.

The rate is calcuated according to the formula:

$$k_{AB} = \phi_{A,0} P(B|\lambda_m) \prod_{i=0}^{m-1} P(\lambda_{i+1} | \lambda_i)$$

where $\phi_{A,0}$ is the flux from state A through its innermost interface, $P(B|\lambda_m)$ is the conditional transition probability (the probability that a path which crosses the interface at $\lambda_m$ ends in state B), and $\prod_{i=0}^{m-1} P(\lambda_{i+1} | \lambda_i)$ is the total crossing probability. We can look at each of these terms individually.

### Total crossing probability

In [ ]:
stateA = storage.volumes["A"]
stateB = storage.volumes["B"]
stateC = storage.volumes["C"]

In [ ]:
tcp_AB = mstis.transitions[(stateA, stateB)].tcp
tcp_AC = mstis.transitions[(stateA, stateC)].tcp
tcp_BC = mstis.transitions[(stateB, stateC)].tcp
tcp_BA = mstis.transitions[(stateB, stateA)].tcp
tcp_CA = mstis.transitions[(stateC, stateA)].tcp
tcp_CB = mstis.transitions[(stateC, stateB)].tcp

plt.plot(tcp_AB.x, tcp_AB)
plt.plot(tcp_CA.x, tcp_CA)
plt.plot(tcp_BC.x, tcp_BC)
plt.plot(tcp_AC.x, tcp_AC) # same as tcp_AB in MSTIS

We normally look at these on a log scale:

In [ ]:
plt.plot(tcp_AB.x, np.log(tcp_AB))
plt.plot(tcp_CA.x, np.log(tcp_CA))
plt.plot(tcp_BC.x, np.log(tcp_BC))

### Flux

Here we also calculate the flux contribution to each transition. The flux is calculated based on 

In [ ]:
import pandas as pd
flux_matrix = pd.DataFrame(columns=mstis.states, index=mstis.states)
for state_pair in mstis.transitions:
    transition = mstis.transitions[state_pair]
    flux_matrix.set_value(state_pair[0], state_pair[1], transition._flux)

flux_matrix

### Conditional transition probability

In [ ]:
outer_ctp_matrix = pd.DataFrame(columns=mstis.states, index=mstis.states)
for state_pair in mstis.transitions:
    transition = mstis.transitions[state_pair]
    outer_ctp_matrix.set_value(state_pair[0], state_pair[1], transition.ctp[transition.ensembles[-1]])    

outer_ctp_matrix

In [ ]:
ctp_by_interface = pd.DataFrame(index=mstis.transitions)
for state_pair in mstis.transitions:
    transition = mstis.transitions[state_pair]
    for ensemble_i in range(len(transition.ensembles)):
        ctp_by_interface.set_value(
            state_pair, ensemble_i,
            transition.conditional_transition_probability(
                storage.steps,
                transition.ensembles[ensemble_i]
        ))
    
    
ctp_by_interface  

## Path ensemble properties

In [ ]:
hists_A = mstis.transitions[(stateA, stateB)].histograms
hists_B = mstis.transitions[(stateB, stateC)].histograms
hists_C = mstis.transitions[(stateC, stateB)].histograms

### Interface crossing probabilities

We obtain the total crossing probability, shown above, by combining the individual crossing probabilities of 

In [ ]:
for hist in [hists_A, hists_B, hists_C]:
    for ens in hist['max_lambda']:
        normalized = hist['max_lambda'][ens].normalized()
        plt.plot(normalized.x, normalized)

In [ ]:
# add visualization of the sum

In [ ]:
for hist in [hists_A, hists_B, hists_C]:
    for ens in hist['max_lambda']:
        reverse_cumulative = hist['max_lambda'][ens].reverse_cumulative()
        plt.plot(reverse_cumulative.x, reverse_cumulative)

In [ ]:
for hist in [hists_A, hists_B, hists_C]:
    for ens in hist['max_lambda']:
        reverse_cumulative = hist['max_lambda'][ens].reverse_cumulative()
        plt.plot(reverse_cumulative.x, np.log(reverse_cumulative))

### Path length histograms

In [ ]:
for hist in [hists_A, hists_B, hists_C]:
    for ens in hist['pathlength']:
        normalized = hist['pathlength'][ens].normalized()
        plt.plot(normalized.x, normalized)

In [ ]:
for ens in hists_A['pathlength']:
    normalized = hists_A['pathlength'][ens].normalized()
    plt.plot(normalized.x, normalized)

## Sampling properties

The properties we illustrated above were properties of the path ensembles. If your path ensembles are sufficiently well-sampled, these will never depend on how you sample them.

But to figure out whether you've done a good job of sampling, you often want to look at properties related to the sampling process. OPS also makes these very easy.

### Move scheme analysis

In [ ]:
scheme = storage.schemes[0]

In [ ]:
scheme.move_summary(storage.steps)

In [ ]:
scheme.move_summary(storage.steps, 'shooting')

In [ ]:
scheme.move_summary(storage.steps, 'minus')

In [ ]:
scheme.move_summary(storage.steps, 'repex')

In [ ]:
scheme.move_summary(storage.steps, 'pathreversal')

### Replica exchange sampling

See the notebook `repex_networks.ipynb` for more details on tools to study the convergence of replica exchange. However, a few simple examples are shown here. All of these are analyzed with a separate object, `ReplicaNetwork`.

In [ ]:
repx_net = paths.ReplicaNetwork(scheme, storage.steps)

#### Replica exchange mixing matrix

In [ ]:
repx_net.mixing_matrix()

#### Replica exchange graph

The mixing matrix tells a story of how well various interfaces are connected to other interfaces. The replica exchange graph is essentially a visualization of the mixing matrix (actually, of the transition matrix -- the mixing matrix is a symmetrized version of the transition matrix).

Note: We're still developing better layout tools to visualize these.

In [ ]:
repxG = paths.ReplicaNetworkGraph(repx_net)
repxG.draw('spring')

#### Replica exchange flow

Replica flow is defined as ***TODO***

Flow is designed for calculations where the replica exchange graph is linear, which ours clearly is not. However, we can define the flow over a subset of the interfaces.

### Replica move history tree

In [ ]:
import openpathsampling.visualize as vis
reload(vis)
from IPython.display import SVG

In [ ]:
tree = vis.PathTree(
    storage.steps[0:200],
    vis.ReplicaEvolution(replica=2, accepted=False)
)

SVG(tree.svg())

In [ ]:
decorrelated = tree.generator.decorrelated
print "We have " + str(len(decorrelated)) + " decorrelated trajectories."

### Visualizing trajectories

In [ ]:
from toy_plot_helpers import ToyPlot
background = ToyPlot()
background.contour_range = np.arange(-1.5, 1.0, 0.1)
background.add_pes(storage.engines[0].pes)

In [ ]:
xval = paths.CV_Function("xval", lambda snap : snap.xyz[0][0])
yval = paths.CV_Function("yval", lambda snap : snap.xyz[0][1])
live_vis = paths.LiveVisualization(mstis, xval, yval, [-1.0, 1.0], [-1.0, 1.0])
live_vis.background = background.plot()

In [ ]:
live_vis.draw_samples(list(tree.samples))

## Histogramming data (TODO)

In [ ]:
#! skip
# The skip directive tells our test runner not to run this cell
import time
max_step = 10
for step in storage.steps[0:max_step]:
    live_vis.draw_ipynb(step)
    time.sleep(0.1)